In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim

import dlc_practical_prologue as prologue


In [2]:
pairs = prologue.generate_pair_sets(1000)

train_input=pairs[0].cuda()
train_target=pairs[1].cuda()
train_classes=pairs[2].cuda()
test_input=pairs[3].cuda()
test_target=pairs[4].cuda()
test_classes=pairs[5].cuda()

# train_target=prologue.convert_to_one_hot_labels(train_input,train_target)

In [3]:
class ConvNet(nn.Module):
    '''
    Input size :            2x14x14
    nn.Conv2d(2, 32, k=3) : 32x12x12
    F.max_pool2d(k=2) :     32x6x6
    nn.Conv2d(32, 64, k=3): 64x4x4
    F.max_pool2d(k=2):      64x2x2
    '''
    def __init__(self, nb_hidden):
        super().__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x

In [4]:
def train_model(model, train_input, train_target):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = 1e-3)
    nb_epochs = 50
    mini_batch_size= 100

    for e in range(nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


In [5]:
def compute_nb_errors(model, data_input, data_target):

    nb_data_errors = 0
    mini_batch_size= 100
    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output, 1)
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

In [6]:
train_E=[]
test_E=[]

for k in range(5):
    model=ConvNet(200).cuda()
    train_model(model,train_input,train_target)
    train_E.append(compute_nb_errors(model, train_input, train_target) / train_input.size(0) * 100)
    test_E.append(compute_nb_errors(model, test_input, test_target) / test_input.size(0) * 100)
    print('train_error {:.02f}% test_error {:.02f}%'.format(train_E[-1],test_E[-1]))

train_error 21.80% test_error 1.50%
train_error 22.90% test_error 1.70%
train_error 21.20% test_error 7.10%
train_error 20.30% test_error 1.90%
train_error 23.30% test_error 2.00%


In [7]:
test_E

[21.8, 22.900000000000002, 21.2, 20.3, 23.3]